In [3]:
import pandas as pd

In [4]:
vi_df = pd.read_excel('../bieuthue.xlsx', sheet_name="VI", header = 1, converters={'dashes':int, 'hs': str, '_nest_parent_':str}).fillna("")[['hs', 'title', 'notes', 'general', '_nest_parent_']]
vi_df = vi_df.rename(columns={'title': 'description'})
vi_section_chapter = vi_df.iloc[17262:,:]


en_df = pd.read_excel('../bieuthue.xlsx', sheet_name="EN", header = 1, converters={'dashes':int, 'hs': str, '_nest_parent_':str}).fillna("")[['hs', 'title', 'notes', 'general', '_nest_parent_']]
en_df = en_df.rename(columns={'title': 'en_description', 'notes': 'en_notes', 'general': 'en_general'})
en_section_chapter = en_df.iloc[17284:,:]


section_chapter = pd.merge(vi_section_chapter, en_section_chapter, on=['hs', '_nest_parent_'], how="left").fillna("")

ko_df = pd.read_excel('../bieuthue.xlsx', sheet_name="KO", header = 1, converters={'dashes':int, 'hs': str, '_nest_parent_':str}).fillna("")[['hs', 'title', 'notes', 'general', '_nest_parent_']]
ko_df = ko_df.rename(columns={'title': 'ko_description', 'notes': 'ko_notes', 'general': 'ko_general'})
ko_section_chapter = ko_df.iloc[17262:,:]

section_chapter = pd.merge(section_chapter, ko_section_chapter, on=['hs', '_nest_parent_'], how="left").fillna("")

section_chapter.to_excel('section_chapter.xlsx', index=False)

In [5]:
has_nest_parent = pd.read_excel("has_nest_parent.xlsx", converters={'dashes':int, 'hs': str, '_nest_parent_':str}).fillna("")
has_nest_parent['id'] = has_nest_parent.index + 1
has_nest_parent['hs45'] = has_nest_parent['hs'].shift(45).fillna("")
has_nest_parent['hs43'] = has_nest_parent['hs'].shift(43).fillna("")
has_nest_parent['hs3'] = has_nest_parent['hs'].shift(3).fillna("")
has_nest_parent['hs1'] = has_nest_parent['hs'].shift(1).fillna("")

In [6]:
vi_df = pd.read_excel('../bieuthue.xlsx', sheet_name="VI", header = 1, converters={'dashes':int, 'hs': str, '_nest_parent_':str}).fillna("")[['hs', 'notes', 'general', '_nest_parent_']]
vi_df['hs45'] = vi_df['hs'].shift(45).fillna("")
vi_df['hs43'] = vi_df['hs'].shift(43).fillna("")
vi_df['hs3'] = vi_df['hs'].shift(3).fillna("")
vi_df['hs1'] = vi_df['hs'].shift(1).fillna("")

In [7]:
vi_df_with_nest_parent = pd.merge(has_nest_parent, vi_df, on=['hs', '_nest_parent_', 'hs1', 'hs43', 'hs3', 'hs45'], how='left').fillna('')
vi_df_with_nest_parent = vi_df_with_nest_parent.drop_duplicates(subset='id')
# vi_df_with_nest_parent.to_excel("vi_df_with_nest_parent.xlsx", index=False)


In [8]:
en_df = pd.read_excel('../bieuthue.xlsx', sheet_name="EN", header = 1, converters={'dashes':int, 'hs': str, '_nest_parent_':str}).fillna("")[['hs', 'notes', 'general', '_nest_parent_']]
en_df = en_df.rename(columns={'notes':'en_notes', 'general':'en_general'})
en_df['hs45'] = en_df['hs'].shift(45).fillna("")
en_df['hs43'] = en_df['hs'].shift(43).fillna("")
en_df['hs3'] = en_df['hs'].shift(3).fillna("")
en_df['hs1'] = en_df['hs'].shift(1).fillna("")

In [9]:
en_vi_df_with_nest_parent = pd.merge(vi_df_with_nest_parent, en_df, on=['hs', '_nest_parent_', 'hs1', 'hs43', 'hs3', 'hs45'], how='left').fillna('')
en_vi_df_with_nest_parent = en_vi_df_with_nest_parent.drop_duplicates(subset='id')
# en_vi_df_with_nest_parent.to_excel('en_vi_df_with_nest_parent.xlsx', index=False)

In [10]:
ko_df = pd.read_excel('../bieuthue.xlsx', sheet_name="KO", header = 1, converters={'dashes':int, 'hs': str, '_nest_parent_':str}).fillna("")[['hs', 'title', 'notes', 'general', '_nest_parent_']]
ko_df = ko_df.rename(columns = {'title': 'ko_description', 'notes': 'ko_notes', 'general': 'ko_general'})
ko_df['hs45'] = ko_df['hs'].shift(45).fillna("")
ko_df['hs43'] = ko_df['hs'].shift(43).fillna("")
ko_df['hs3'] = ko_df['hs'].shift(3).fillna("")
ko_df['hs1'] = ko_df['hs'].shift(1).fillna("")

In [11]:
ko_en_vi_with_nest_parent = pd.merge(en_vi_df_with_nest_parent, ko_df, on=['hs', '_nest_parent_', 'hs1', 'hs43', 'hs3', 'hs45'], how='left').fillna('')
ko_en_vi_with_nest_parent = ko_en_vi_with_nest_parent.drop_duplicates(subset='id') # maybe lost wrong data
df = ko_en_vi_with_nest_parent.append(section_chapter, ignore_index=True).reset_index(drop=True)
df['id'] = df.index + 1

C:\Users\thanh\AppData\Local\Temp\ipykernel_11188\3681262229.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = ko_en_vi_with_nest_parent.append(section_chapter, ignore_index=True).reset_index(drop=True)


In [12]:
def fix_missing_ko_description(row):
    if row['ko_description'] == '':
        try:
            lookup = ko_df.loc[ko_df['hs'] == row['hs']]
            row['ko_description'] = lookup.iloc[0]['ko_description']
        except:
            return row
    return row

fixed_df = df.apply(fix_missing_ko_description, axis=1)
# fixed_df.to_excel('fixed_df.xlsx', index=False)

In [13]:
regulation_id = pd.read_excel('../bieuthue.xlsx', sheet_name="Chỉnh regulation tại cột vàng").fillna("")[['hs', 'regulation_id']].drop_duplicates(subset='hs')
fixed_df_added_regulation_id = pd.merge(fixed_df, regulation_id, on='hs', how='left', validate="many_to_one").fillna('')
rcep = pd.read_excel('../bieuthue.xlsx', sheet_name="RCEP", header=4)[['hs', 'RCEP_China', 'RCEP_NewZealand', 'RCEP_Japan', 'RCEP_Korea', 'RCEP_Australia', 'RCEP_Asean']]
df = pd.merge(fixed_df_added_regulation_id, rcep, on='hs', how='left', validate="many_to_one").fillna("")

In [14]:
df = df.drop(columns = ['hs1', 'hs45', 'hs43', 'hs3'])
cols = df.columns.to_list()
disire_cols = ['description', 'en_description', 'ko_description', 'notes', 'en_notes', 'ko_notes', 'general', 'en_general', 'ko_general']
cols = cols[:3] + disire_cols + cols[5:-14] + cols[-7:]
df[cols].to_excel('result.xlsx', index = False)